In [107]:
import numpy as np
import pandas as pd
import pandas_datareader as web
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

from pypfopt import discrete_allocation
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.cla import CLA
from pypfopt import plotting

              sp500
DATE               
2015-01-01      NaN
2015-01-02  2058.20
2015-01-05  2020.58
2015-01-06  2002.61
2015-01-07  2025.90
...             ...
2020-06-01  3055.73
2020-06-02  3080.82
2020-06-03  3122.87
2020-06-04  3112.35
2020-06-05  3193.93

[1417 rows x 1 columns]


In [79]:
tickers = ['BSX','AES','BRK-B','SEE','QQQ','SPY']
thelen = len(tickers)
price_data = []
for ticker in range(thelen):
    prices = web.DataReader(tickers[ticker], start='2015-01-01', end = '2020-06-06', data_source='yahoo')
    price_data.append(prices.assign(ticker=ticker)[['Adj Close']])
df_stocks = pd.concat(price_data, axis=1)
df_stocks.columns=tickers
df_stocks

,BSX,AES,BRK-B,SEE,QQQ,SPY
Date,,,,,,
2014-12-31,13.250000,10.683700,150.149994,38.254536,97.376846,179.880402
2015-01-02,13.220000,10.629392,149.169998,38.497959,97.084496,179.784119
2015-01-05,13.810000,10.326804,147.000000,37.947994,95.660385,176.537277
2015-01-06,13.700000,10.094044,146.839996,38.146351,94.377731,174.874512
2015-01-07,14.030000,10.109560,148.880005,38.200439,95.594337,177.053650
...,...,...,...,...,...,...
2020-06-01,37.500000,11.960445,183.839996,31.503242,231.826096,296.821289
2020-06-02,37.540001,12.494140,185.940002,31.522673,233.381119,299.278992
2020-06-03,37.910000,12.989712,190.820007,32.659241,234.431000,303.261932


In [80]:
nullin_df = pd.DataFrame(df_stocks,columns=tickers)
print(nullin_df.isnull().sum())

BSX      0
AES      0
BRK-B    0
SEE      0
QQQ      0
SPY      0
dtype: int64


In [81]:
mu = expected_returns.mean_historical_return(df_stocks)
Sigma = risk_models.sample_cov(df_stocks)

In [100]:
ef = EfficientFrontier(mu, Sigma, weight_bounds=(0,1))
sharpe_pfolio=ef.min_volatility()
sharpe_pwt=ef.clean_weights()
print(sharpe_pwt)

OrderedDict([('BSX', 0.0), ('AES', 0.0), ('BRK-B', 0.08013), ('SEE', 0.05241), ('QQQ', 0.0), ('SPY', 0.86746)])


In [101]:
latest_prices = discrete_allocation.get_latest_prices(df_stocks)
print(latest_prices)
allocation_minv, rem_minv = discrete_allocation.DiscreteAllocation(sharpe_pwt, latest_prices, total_portfolio_value=10000).lp_portfolio()
print(allocation_minv)
print(rem_minv)

BSX       38.830002
AES       13.628237
BRK-B    200.660004
SEE       34.289272
QQQ      237.402374
SPY      310.217407
Name: 2020-06-05 00:00:00, dtype: float64
{'BRK-B': 4, 'SEE': 14, 'SPY': 28}Long-step dual simplex will be used

31.22277069091797


In [102]:
df = latest_prices
cost = 0
for index, val in allocation_minv.items():
    cost = cost + df[index]*val
print(cost)

9968.777229309082


In [114]:
sp500_df = web.DataReader('sp500', start ='2015-01-02', end = '2020-06-06', data_source='fred')
print(sp500_df.head())

              sp500
DATE               
2015-01-02  2058.20
2015-01-05  2020.58
2015-01-06  2002.61
2015-01-07  2025.90
2015-01-08  2062.14


In [119]:
sp500_df.loc['2015-01-02']

sp500    2058.2
Name: 2015-01-02 00:00:00, dtype: float64